In [ ]:
from dolfin import *
import numpy as np

In [ ]:
# Input parameter
# Material (biophysical) parameter
D = 50
chi = 0
M = 1
sig_PH = 0.4
sig_HN = 0.2

lambda_P =  500
lambda_Ph = 250
lambda_A = 100
lambda_PH = 250
lambda_HN = 250


# Phase field parameter
epsilon = 0.01

# Domain
R = 0.01
pts = [0, 0, 1, 1]
n = 100 
sig_D = 1.0
sig_0 = 1.0

# Time
dt = 5.0e-06
theta = 0.5
nT = 2000
T = dt*nT

In [ ]:
# Define initial condition as a subclass of UserExpression
class InitialCondition(UserExpression):
    def eval(self, value, x):
        if ((x[0]-(pts[0]+pts[2])/2)**2 + (x[1]-(pts[1]+pts[3])/2)**2) - R >= DOLFIN_EPS:
            value[0] = 0.0   # value phi_T
            value[1] = 0.0   # value phi_H
            value[2] = 0.0   # value phi_N
            value[3] = 0.0   # value mu
            value[4] = sig_0 # value sig
        else:
            value[0] = 1.0   # value phi_T
            value[1] = 0.0   # value phi_H
            value[2] = 0.0   # value phi_N
            value[3] = 0.0   # value mu
            value[4] = sig_0 # value sig
    def value_shape(self):
        return(5,)

In [ ]:
# Create mesh and build function space
mesh = RectangleMesh(Point(pts[0], pts[1]), Point(pts[2], pts[3]), n, n)
P1 = FiniteElement('P', triangle, 1)
element = MixedElement([P1, P1, P1, P1, P1])
ME = FunctionSpace(mesh, element)

In [ ]:
# Define trial and test functions
du = TrialFunction(ME)
q, s, t, v, w = TestFunctions(ME)

In [ ]:
# Define solutions (current and previous) as function of ME
u = Function(ME)
u0 = Function(ME)

# Split mixed functions
dphiT, dphiH, dphiN, dmu, dsig = split(du)
phiT, phiH, phiN, mu, sig = split(u)
phiT0, phiH0, phiN0, mu0, sig0 = split(u0)

In [ ]:
# Create instance of initial conditions and interpolate
u_init = InitialCondition(degree=1)
u_init_interp = interpolate(u_init, ME)
u.vector()[:] = u_init_interp.vector()[:]
u0.vector()[:] = u_init_interp.vector()[:]

In [ ]:
# Create Dirchlet boundary condition for nutrients only
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(ME.sub(4), sig_D, boundary)

In [ ]:
# Compute the chemical potential df/dphi
phiT = variable(phiT)
f = 100*phiT**2 *(1-phiT)**2
dfdphiT = diff(f, phiT)

In [ ]:
# Compute the Heaviside functions for the different phase transitions 
sig = variable(sig)
heaviPH = conditional(gt(sig-sig_PH,0),0,1)
heaviHP = conditional(gt(sig_PH-sig,0),0,1)
heaviHN = conditional(gt(sig-sig_HN,0),0,1)

In [ ]:
# Time stepping variable
mu_mid = (1.0-theta)*mu0 + theta*mu

In [ ]:
# State weak formulation of the governing equations
dx = Measure('dx', mesh)

L0 = (phiT-phiT0)*q*dx + phiT**2*M*dt*mu_mid*q*dx - dt*lambda_P*sig*(phiT-phiH-phiN)*q*dx + dt*lambda_A*(phiT-phiN)*q*dx +\
     dt*lambda_PH*heaviPH*(phiT-phiH-phiN)*q*dx - dt*lambda_PH*heaviHP*phiH*q*dx
L1 = (phiH-phiH0)*s*dx + phiH**2*M*dt*mu_mid*s*dx + dt*lambda_A*phiH*s*dx - dt*lambda_PH*heaviPH*(phiT-phiH-phiN)*s*dx + \
     dt*lambda_PH*heaviHP*phiH*s*dx + dt*lambda_HN*heaviHN*phiH*s*dx
L2 = (phiN-phiN0)*t*dx + phiN**2*M*dt*mu_mid*t*dx - dt*lambda_HN*heaviHN*phiH*t*dx 
L3 = mu*v*dx - dfdphiT/epsilon*v*dx - dot(grad(phiT), grad(v))*dx
L4 = (sig-sig0)*w*dx + dt*dot(grad(w), grad(sig))*dx + dt*lambda_P*sig*(phiT-phiH-phiN)*w*dx + dt*lambda_Ph*sig*phiH*w*dx

L = L0 + L1 + L2 + L3 + L4

In [ ]:
# Compute directional derivative about u in the direction of du (Jacobian)
a = derivative(L, u, du)

In [ ]:
# Create output file and specify save location
file1 = File("FourPhaseAC/phi_T.pvd", "compressed")
file2 = File("FourPhaseAC/phi_H.pvd", "compressed")
file3 = File("FourPhaseAC/phi_N.pvd", "compressed")
file4 = File("FourPhaseAC/sig.pvd", "compressed")
u.rename("phasefield", "u")

In [ ]:
# Compute the transient problem until terminal time T and write to file in 
# every time step
t = 0
while (t < T):
    t += dt
    u0.vector()[:] = u.vector()
    solve(L==0, u, bcs=bc, J=a)
    
    
    file1 << (u.split()[0], t)
    file2 << (u.split()[1], t)
    file3 << (u.split()[2], t)
    file4 << (u.split()[4], t)